In [3]:
import shutil
import os
import glob
import subprocess


In [2]:
SOURCE_GUEST_PATH = "./0_guest"
SOURCE_HOST_PATH = "./1_host"
SETUP_SCRIPT_PATH = './setup.py'
RELEASE_FOLDER_PATH = "./release"
RELEASE_BUILD_PATH = f'{RELEASE_FOLDER_PATH}/build'
RELEASE_STABLE_PATH = f'{RELEASE_FOLDER_PATH}/stable'
RELEASE_HOST_PATH = f"{RELEASE_STABLE_PATH}/1_host"
RELEASE_GUEST_PATH = f"{RELEASE_STABLE_PATH}/0_guest"

In [3]:
MAIN_README="""
# description
0_guest for virtual machine, contains readme
1_host for host machine, contains readme

# How to install:
- shortly, install python 3.8.10, you can download it from https://www.python.org/downloads/release/python-3810/ on host and guest(virtual machine)
- guest/readme.txt for virtual

# how to run
- run virtual machine
- start poe, hud, start.bat on your virtual machine, copy ip from your virtual machine
- guest ready
- input virtual machine ip into the .bat file, dont forget to input key
- run .bat

# troubleshooting
repeating msg when running .bat on host: ImportError: DLL load failed while importing _path: The specified module could not be found.
install VC 2015 Redistributable https://www.microsoft.com/en-us/download/confirmation.aspx?id=48145

# Answers
Q: can i run it without virtual machine?
A: Yes, you may try to run files from 0_guest locally and launch files from 1_host, but 0_guest will control your mouse, so youll face troubles 
in stopping the script

Q: t16-t17 strats?
A: t17 maps: if the bot will finish map in 10 minutes, you'd need 6*16 maps_per_hour*hours a day, not quite sure if you can get it

Q: it ignores *SOME MAP* or ignores boss in *SOME MAP*
A: some maps are in ignore list since they are complicated, some bossrooms in maps are ignored since the bossfight logic is complicated

Q: only mapper?
A: 3d/hour simulacrum abandoned, heist needs tons of stash tabs and profits are shady, aqueduct - bannable, questbot - in process, some bosses are not supported yet, rewards in town, etc.

Q: only venom gyre?
A: also cold dot elementalist(nerfed), lightning arrow flicker, detonate dead elementalist, penance brand pf with flasks, cast on stun pf, 
deadeye bow and wander, currently making corrupting fever and ea ballistas

Q: how can i make my own strategy?
A: at the current stage, it's complicated, but if youre familiar with json or dictionaries, you may edit 1_host/modules/mapper_config.py

Q: why making personal strategies is so complicated?
A: im working on a launcher atm, but there are different priorities

Q: contacts?
A: telegram: https://t.me/Shmeshhlai
"""
HOST_README="""
# How to install:
- install python 3.8.10, you can download it from https://www.python.org/downloads/release/python-3810/ , "add to PATH" during python installation
- install VC 2015 Redistributable https://www.microsoft.com/en-us/download/confirmation.aspx?id=48145
- after you did install python, run utils/2_install_python_libs.bat, itll install all the python dependencies

# Important:
- be sure to disable firewall and disable turning off the monitor, if not sure how to do that, run utils/0_preset.bat

# How to run:
- open run_mapper.bat with file editor
- replace "set unique_id=<YOUR_KEY>" with provided key e.g. "set remote_ip=8dfa109eas45346c4f99"
- replace "set remote_ip=<YOUR_IP>" with ip from 0_guest from your virtual machine e.g. "set unique_id=192.168.1.1"
- replace "set predefined_strategy=<YOUR_MAPPING_STRATEGY_HERE>" with strategy from list below e.g. "set predefined_strategy=atlas_explorer"

# strategy descriptions:
- atlas_explorer: runs uncompleted maps for atlas points, transmute/alch/vaal - ok, kirak missions - broken atm

- low_tier_incursion: runs incursion with scarabs and finishing the map till end
- low_tier_incursion_rush: runs incursion with scarabs, if done all incursions and have decent amount of maps, will finish the map, else finishing the map till end

- low_tier_deli_incursion: farm splinters
- low_tier_deli_incursion_with_scarab: farm splinters + temples
- high_tier_deli_incursion: farm splinters
- high_tier_deli_incursion_with_scarab: farm splinters + temples

- low_tier_beast_no_scarabs: runs maps for bestiary without scarabs
- low_tier_beast_with_scarabs: runs maps for bestiary with herd herd duplicate scarabs

- essence_early: runs white maps for essences
- essence_early_yellow: runs preferably yellow maps for essences
- lowtier_ess_expensive_ascent_force: runs yellow tier essences with ess,ess,ascent,ess scarabs
- lowtier_ess_expensive_ascent_force_no_wailing: runs yellow tier essences with ess,ess,ascent,ess scarabs and skips essences containing only wailing
- lowtier_ess_expensive_ascent_force_no_wailing_map_device: runs yellow tier essences with ess,ess,ascent,ess scarabs and map device mod and skips essences containing only wailing

# other builds:
builds = [
  'LightningArrowLightningWarp',
  'ColdDotElementalist',
  "DetonateDeadMapper",
  "DetonateDeadSimulacrum",
  "VenomGyre",
  "FrenzyFrostblink",
  "PenanceBrandPf",
  "CastOnStunPf",
  "HitBasedDeadeye",
  "CfKineticBlastChampion",
  "EaBallistasEle",
  "ScourgeArrowPf",
]

only vg is tested on 3.25

"""

RUN_BAT_SAMPLE = """
set machine=s2
set debug=0
set password=None
set build_name=VenomGyre
set unique_id=<YOUR_KEY_HERE>
set remote_ip=<REMOTE_IP_HERE>
set predefined_strategy=<YOUR_MAPPING_STRATEGY_HERE>
TITLE %machine% %remote_ip% %predefined_strategy% %build_name%
:infinity_loop
python launch.py {'script':'maps','REMOTE_IP':'%remote_ip%','unique_id':'%unique_id%','password':'%password%','force_reset_temp':False,'custom_strategy':'','predefined_strategy':'%predefined_strategy%','build':'%build_name%'}
goto :infinity_loop
"""


HOST_REQUIREMENTS="""
matplotlib==3.4.3
matplotlib-inline==0.1.3
numpy==1.23.0
opencv-contrib-python==4.5.5.64
opencv-python==4.5.5.64
pyastar2d==1.0.6
requests==2.26.0
"""
HOST_MODULES = [
  "matplotlib==3.4.3",
  "matplotlib-inline==0.1.3",
  "numpy==1.23.0",
  "opencv-contrib-python==4.5.5.64",
  "opencv-python==4.5.5.64",
  "pyastar2d==1.0.6",
  "requests==2.26.0",
]
GUEST_README="""
# How to install:
- install python 3.8.10, you can download it from https://www.python.org/downloads/release/python-3810/ , dont forget to check on "add to PATH" during python installation
- install .NET SDK 8.0 x64 https://dotnet.microsoft.com/en-us/download/dotnet/8.0
- install VC 2015 Redistributable https://www.microsoft.com/en-us/download/confirmation.aspx?id=48145
- after you did install python, run utils/2_install_python_libs.bat, itll install all the python dependencies
- exileapi can be downloaded at https://www.ownedcore.com/forums/mmo/path-of-exile/poe-bots-programs/1000594-exileapi-3-23-beyond-plugin-framework.html

# Important:
- be sure to disable firewall and disable turning off the monitor, if not sure how to do that, run utils/0_preset.bat
- be sure to set the game resolution to 1024x768, otherwise it may cause unexpected behaiviour
- if you have vpn, make sure that you dont pass connections through ports 50006,50007
- be sure to use custom loot filter for mapper, at the current state it picks everything what drops on maps, except for portal scrolls, cos there is some logic behind it 
- lootfilters can be placed at Users\<your_user>\Documents\My Games\Path of Exile
- you can also use my production_Config.ini, and place it together with filters


# How to run:
- run Path of Exile
- run ExileApi with ShareData plugin
- run start.bat
- YOUR_IP from line *host:port <YOUR_IP>:50007* will be used in bat files on host"""
GUEST_REQUIREMENTS="""
numpy==1.23.0
pywin32==304
opencv-python==4.5.5.64
requests==2.26.0
"""
GUEST_MODULES = [
  "numpy==1.23.0",
  "pywin32==304",
  "opencv-python==4.5.5.64",
  "requests==2.26.0",
]
DISABLE_FIREWALL_BAT="""
NetSh Advfirewall set allprofiles state off
powercfg -change -monitor-timeout-ac 0
pause
"""
INSTALL_REQUIREMENTS="""
pip install -r requirements.txt
pause
"""
EXILEAPI_PLUGIN_README="""
place this folder into \Plugins\Compiled folder of your exileapi
"""
GUEST_BAT_CONTENT = """
:infinity_loop
python vm_host_main_socket.py
goto :infinity_loop
"""




In [1]:
def generateModuleTemplate(main_module:str, module_requirements:list):
  return f""" 
import sys;
import pkg_resources

if sys.version.split(" ")[0] != "3.8.10":
  sys.exit("Python version isnt 3.8.10 reinstall python from https://www.python.org/downloads/release/python-3810/")

# installed = set(pkg.key for pkg in pkg_resources.working_set)
# if "numpy" not in installed: sys.exit('run other/2_install_python_libs.bat')

if "numpy" not in set(map(lambda lib: lib.key, pkg_resources.working_set)):
  import subprocess
  python = sys.executable
  required = [{'"'+'", "'.join(module_requirements)+'"'}]
  subprocess.check_call([python, '-m', 'pip', 'install', *required], stdout=subprocess.DEVNULL)
  sys.exit("installed missing dependancies, restart")
__import__("{main_module}")
"""
def execute(cmd_command, directory:str = None):
  print(f'executing {cmd_command}')
  process  = subprocess.Popen(cmd_command.split(), shell=False, stdout=subprocess.PIPE,stderr=subprocess.STDOUT, cwd=directory)
  while True:   # Could be more pythonic with := in Python3.8+
      line = process.stdout.readline()
      if not line and process.poll() is not None:
          break
      print(line.decode(), end='')
def writeFile(path, content):
  f = open(path, "w+")   # 'r' for reading and 'w' for writing
  f.write(content)    # Write inside file 
  f.close()  
def compileFolder(src_dir, src_files, build_dir, output_dir, main_module = None):
  for file in glob.glob(src_dir+"/*.py"):
    filename = file.split("\\")[1]
    if not filename in src_files:
      continue
    print(filename)
    shutil.copy(file,build_dir)
  shutil.copy(SETUP_SCRIPT_PATH,build_dir)
  command = "python setup.py build_ext --build-lib=./generated"
  execute(command, build_dir)
  shutil.copytree(build_dir+"/generated",output_dir, dirs_exist_ok=True)
  if main_module:
    template = f"""
    __import__("{main_module}")
    """
    writeFile(output_dir+"/main.py", template)


In [5]:
# remove release folder
try:
  shutil.rmtree(RELEASE_FOLDER_PATH)
except FileNotFoundError:
  pass


In [6]:
# create release folder
# create 0_guest 1_host folder in release folder
os.mkdir(RELEASE_FOLDER_PATH)
os.mkdir(RELEASE_BUILD_PATH)
os.mkdir(RELEASE_STABLE_PATH)

os.mkdir(RELEASE_GUEST_PATH)
os.mkdir(RELEASE_GUEST_PATH+"/other")
os.mkdir(RELEASE_GUEST_PATH+"/loot_filters")

os.mkdir(RELEASE_HOST_PATH)
os.mkdir(RELEASE_HOST_PATH+"/other")
os.mkdir(RELEASE_HOST_PATH+"/utils")
os.mkdir(RELEASE_HOST_PATH+"/temp")


In [7]:
# copy lootfilters and production.ini into 0_guest/loot_filters
loot_filter_files_save_loc = f"{RELEASE_GUEST_PATH}/loot_filters"
loot_filter_files = glob.glob(f"{SOURCE_GUEST_PATH}/loot_filters/*.filter")
loot_filter_files.extend(glob.glob(f"{SOURCE_GUEST_PATH}/loot_filters/production_Config.ini"))
for file in loot_filter_files:
  shutil.copy(file,loot_filter_files_save_loc)


In [8]:
hud_src_dir = f"{SOURCE_GUEST_PATH}/Python28"
hud_plugin_src_dir = hud_src_dir + "/Plugins/Compiled/ShareData"
shutil.copytree(hud_src_dir,f"{RELEASE_GUEST_PATH}/hud_with_plugin/RENAME_ME")
#TODO rename java.exe to RENAME_ME.EXE 
shutil.copytree(hud_plugin_src_dir,f"{RELEASE_GUEST_PATH}/hud_only_plugin/ShareData")

'./release/stable/0_guest/hud_only_plugin/ShareData'

In [9]:
# guest
src_dir = SOURCE_GUEST_PATH
src_files = [
  "utils.py",
  "vm_host_main_socket.py",
]
build_dir = RELEASE_BUILD_PATH+'/guest'
output_dir = RELEASE_GUEST_PATH
os.mkdir(RELEASE_BUILD_PATH+"/guest")
compileFolder(src_dir, src_files, build_dir,output_dir)
# host
src_dir = SOURCE_HOST_PATH
src_files = [
  # "launcher.py",
  "maps.py",
  # "aqueduct.py",
  "quest.py",
  "launch.py",
]
build_dir = RELEASE_BUILD_PATH+'/host'
output_dir = RELEASE_HOST_PATH
os.mkdir(RELEASE_BUILD_PATH+"/host")
compileFolder(src_dir, src_files, build_dir,output_dir)
# host_utils
src_dir = SOURCE_HOST_PATH+"/utils"
src_files = [
  'backend.py',
  'combat.py',
  'components.py',
  'constants.py',
  'controller.py',
  'coordinator.py',
  'encounters.py',
  'gamehelper.py',
  'helper_functions.py',
  'launcher.py',
  'loot_filter.py',
  'mapper.py',
  'mover.py',
  'pathing.py',
  'stash_manager.py',
  'temps.py',
  'ui.py',
  'utils.py'
]
build_dir = RELEASE_BUILD_PATH+'/host_utils'
output_dir = RELEASE_HOST_PATH+"/utils"
os.mkdir(RELEASE_BUILD_PATH+"/host_utils")
compileFolder(src_dir, src_files, build_dir,output_dir)


utils.py
vm_host_main_socket.py
executing python setup.py build_ext --build-lib=./generated
going to compile utils.py
going to compile vm_host_main_socket.py
Compiling utils.py because it changed.
Compiling vm_host_main_socket.py because it changed.
[1/2] Cythonizing utils.py
c:\Users\1\AppData\Local\Programs\Python\Python38\lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: c:\poe_bot\release\build\guest\utils.py
  tree = Parsing.p_module(s, pxd, full_module_name)
[2/2] Cythonizing vm_host_main_socket.py
c:\Users\1\AppData\Local\Programs\Python\Python38\lib\site-packages\Cython\Compiler\Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: c:\poe_bot\release\build\guest\vm_host_main_socket.py
  tree = Parsing.p_module(s, pxd, full_module_name)
c:\Users\1\AppData\Local\P

In [10]:
# duplicate requirements
shutil.copy(SOURCE_GUEST_PATH+"/requirements.txt","./0_guest_software_to_install/guest_requirements.txt")
shutil.copy(SOURCE_HOST_PATH+"/requirements.txt","./0_guest_software_to_install/host_requirements.txt")
# main
writeFile(RELEASE_STABLE_PATH+"/readme.txt", MAIN_README)
# guest
writeFile(RELEASE_GUEST_PATH+"/vm_host_main_socket.py", generateModuleTemplate("vm_host_main_socket", GUEST_MODULES))
writeFile(RELEASE_GUEST_PATH+"/readme.txt", GUEST_README)
writeFile(RELEASE_GUEST_PATH+"/start.bat", GUEST_BAT_CONTENT)
writeFile(RELEASE_GUEST_PATH+"/other/0_preset.bat", DISABLE_FIREWALL_BAT)
writeFile(RELEASE_GUEST_PATH+"/other/requirements.txt", GUEST_REQUIREMENTS)
writeFile(RELEASE_GUEST_PATH+"/other/2_install_python_libs.bat", INSTALL_REQUIREMENTS)
# guest/hud_plugin
writeFile(RELEASE_GUEST_PATH+"/hud_only_plugin/readme.txt", EXILEAPI_PLUGIN_README)
# host
# writeFile(RELEASE_HOST_PATH+"/maps.py", generateModuleTemplate("maps", HOST_MODULES))
# writeFile(RELEASE_HOST_PATH+"/aqueduct.py", generateModuleTemplate("aqueduct", HOST_MODULES))
writeFile(RELEASE_HOST_PATH+"/launch.py", generateModuleTemplate("launch", HOST_MODULES))
writeFile(RELEASE_HOST_PATH+"/run_mapper.bat", RUN_BAT_SAMPLE)
writeFile(RELEASE_HOST_PATH+"/readme.txt", HOST_README)
writeFile(RELEASE_HOST_PATH+"/other/0_preset.bat", DISABLE_FIREWALL_BAT)
writeFile(RELEASE_HOST_PATH+"/other/2_install_python_libs.bat", INSTALL_REQUIREMENTS)
writeFile(RELEASE_HOST_PATH+"/other/requirements.txt", HOST_REQUIREMENTS)
shutil.copy(SOURCE_HOST_PATH+"/mapper_configs.py",RELEASE_HOST_PATH)


'./release/stable/1_host\\mapper_configs.py'